<h1 style="text-align: center;color:orange" markdown="1"> DEFI IA  </h1> 

<h3>  </h3> 

 <h2 style="color:blue"> CONTENU </h2>
 
 
 > I. Présentation des méthodes utilisées: **Explication de l'approche** <br>
     
 > II.Meilleur soumission: <br>
     >> II.1 Présentation de l'approche<br>
     >> II.2 Bliblothèques utilisées  <br>
     >> II.3 présentation du code  <br>
     >> II.4 Evaluation des métrics <br>

 > CONCLUSION <br>


 <h5 style="color:blue">I. Notre Expérience dans le challenge: **Explication de l'approche** </h2> 



#### - **Approche Classique**
<br> Nous nous trouvons en face de données textuelles, l'approche classique consiste à transformer le corpus de mots qui est sur un format non structuré dont non adapté pour les modèles de machine learning, pour ce faire nous transformons nos données en données structurées au format compréhensible pour nos modèles de machine learning (**SVM,RandomForest,Naives Bayes,Logistic Regression,XGboost,lightbm**).
Avant la phase de transformation des données en format structuré, les données ont subi un certain nombres de traitements qui sont entre autres:
* La suppression des bruits du corpus telles que : **sites web**,**emails**,**mots non alphanumériques**.
- Méthodes de transformation des données en format structuré utiliser :
* Matrice Tfidf avec utilisations des ngrams
* Création des words vectors en  se basant sur des modèles pré entrainés à l'aide des bilblothèques telles que : **word2vec**,**glove**.

Pour la gestion du problème de classes déséquilibrées nous avions opté pour la méthode de ré-equilibrage
**SMOTE**
**Résultats obtenu**: Après utilisations de toutes ses méthodes classiques, notre meilleur score fut **75.67%**
</br>

#### - **Approche moderne (Etat de l'art en NLP)**
<br> Nous nous sommes tournés également vers les méthodes modernes en  se basant sur les transformateurs
de Hugging Face, l'une des bibliothèques de la PNL qui connaît le plus grand succès actuellement.
avec le méchanisme de transfert learning. Pour ce faire nous utilisons la biblothèque **Ktrain** basé sur 
**Tensorflow** et **Keras** avec le modèle pré-entrainé **distilbert**  qui est une distribution de **Bert**. 

**Pourquoi **Ktrain** et **distilbert**?**:
**Ktrain :**Nous choisissons cette biblothèque **Ktrain** pour sa flexibilité et sa mise en oeuvre.
**distilbert :**Nous optons pour ce modèle car nous l'avons trouvé moins lourd pour la phase d'entrainement
plus ou moins lent.
</br>
**Meilleur résultat obtenu**: **79.96%**

<br> Pour ameliorer les performances de notre algorithme de deep learning, nous nous camptons sur les 
classes ayant les taux de précisions les plus faibles puis faisons une Data augmentation pour accroitre
le nombre d'individus (de descriptions ) présent dans ces classes à l'aide de la méthode **backtranslate**
qui consiste à traduire le texte en une langue donnée puis le retraduire dans sa forme initiale.
</br>
**Résultat obtenu**: **80.10%** (Notre meilleur score sur le leader bord public)
                    

 <h5 style="color:blue">II.Meilleur Soumission </h2> 



 <h5 style="color:blue">II.1 Présentation de l'approche </h2> 



Notre approche pour cette soumission a été essentiellement basé sur l'utilisation des langages models basés sur des transformeurs notamment **distil-bert** car celui-ci a été pour nous le modèle pré-entrainé le plus facile à entrainer. Pour accroitre le pouvoir prédicteurs de notre classifieur basé sur les langages models, nous récensons les classes ayant un taux de prédictions faibles en validation-test puis améliorons la prédictions de ses classes en se basant sur une technique de data-augmentation notamment **backtranslate** ce qui nous permet d'accroitre le nombre d'individus des classes mal prédites.


 <h5 style="color:blue">II.2 Bliblothèques utilisées </h2> 



In [ ]:
%%capture
!pip install ktrain
# version 0.23.2
! pip install transformers
! pip install mosestokenizer
%reload_ext autoreload
%autoreload 2
%matplotlib inline  

In [ ]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import MarianMTModel, MarianTokenizer
import tqdm

 <h5 style="color:blue">II.3 présentation du code</h2> 



In [24]:
#Importation des données
path="C:/Users/konate/Desktop/IA_kaagle/Data"
train_set = pd.read_json(path+"/train.json")
labels = pd.read_csv(path+"/train_label.csv")
categories = pd.read_csv(path+"/categories_string.csv", names=["Libelle", "Category"], header=0)
test_set = pd.read_json(path+"/test.json")

In [11]:
%%time
df_train = train_set.join(labels.set_index("Id"), on = "Id")
df_train = df_train.join(categories.set_index("Category"), on = "Category")


Wall time: 325 ms


Data Augmentation pour améliorer le pouvoir prédictif de notre classifieur sur les classes mal prédites

In [ ]:
# Helper function to download data for a language
def download(model_name):
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

# download model for English -> Romance
tmp_lang_tokenizer, tmp_lang_model = download('Helsinki-NLP/opus-mt-en-ROMANCE')
# download model for Romance -> English
src_lang_tokenizer, src_lang_model = download('Helsinki-NLP/opus-mt-ROMANCE-en')

In [ ]:
def translate(texts, model, tokenizer, language):
    """Translate texts into a target language"""
    # Format the text as expected by the model
    formatter_fn = lambda txt: f"{txt}" if language == "en" else f">>{language}<< {txt}"
    original_texts = [formatter_fn(txt) for txt in texts]

    # Tokenize (text to tokens)
    tokens = tokenizer.prepare_seq2seq_batch(original_texts)

    # Translate
    translated = model.generate(**tokens)

    # Decode (tokens to text)
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)

    return translated_texts

In [ ]:
def back_translate(texts, language_src, language_dst):
    """Implements back translation"""
    # Translate from source to target language
    translated = translate(texts, tmp_lang_model, tmp_lang_tokenizer, language_dst)

    # Translate from target language back to source language
    back_translated = translate(translated, src_lang_model, src_lang_tokenizer, language_src)

    return back_translated

In [ ]:
%%time
df1.loc[:,'description']=back_translate(df1.loc[:,'description'].tolist(),"en","fr")


In [5]:
df=pd.concat([df,df1],axis=0)
df=df.drop(columns=['Unnamed: 0'],axis=1)
df.dropna(inplace=True)
df.Category=df.Category.astype(int)
df_train=pd.concat([df_train.drop(columns=['Id'],axis=1),df],axis=0)

**Echantillonage**

In [13]:
%time
y, categories = df_train['Category'].factorize()
df_train['category_id'] = y
categories = categories.to_list()
# train test split
train_df=df_train.sample(frac=0.8,random_state=200) #random state is a seed value
test_df=df_train.drop(train_df.index)
print(train_df.shape)
print(test_df.shape)

Wall time: 0 ns
(173998, 5)
(43435, 5)


In [16]:
# Choix du modèle pré-entrainés distilbert comportant 66M de parametres 
model_name = 'distilbert-base-uncased'
trans = ktrain.text.Transformer(model_name, maxlen=512, class_names=categories) #maxlen = 512

In [17]:
#Preprocessing à l'aide de ktrain
train_data = trans.preprocess_train(train_df.description.values, train_df.category_id.values)
test_data = trans.preprocess_test(test_df.description.values, test_df.category_id.values)

preprocessing train...
language: en
train sequence lengths:
	mean : 61
	95percentile : 116
	99percentile : 134


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 61
	95percentile : 116
	99percentile : 134


In [18]:
#choisir un classifieur
model = trans.get_classifier()

In [19]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=16)

In [ ]:
 #rechercher le meilleur learning rate (meilleur pas de la descente de gradient)
learner.lr_find(show_plot=True, max_epochs=5)

*Nous formons notre transformers en utilisant l'un des trois taux d'apprentissage recommandés dans l'article BERT: 5e-5, 3e-5 ou 2e-5. Alternativement, l'outil de recherche de taux d'apprentissage ktrain mis dans le chunck précédent  peut être utilisé pour trouver un bon taux d'apprentissage en appelant Learner.lr_find () et Learner.lr_plot (),vu le temps d'entrainement long du classifieur nous optons pour 2 epochs.A voir https://arxiv.org/pdf/1803.09820.pdf*

In [20]:
#Choix du learning rate et du nombre de epocs pour la phase d'apprentissage
learner.fit_onecycle(3e-5, 2)




begin training using onecycle policy with max lr of 3e-05...
Epoch 1/2
10875/10875 [==============================] - 6240s 574ms/step - loss: 0.8003 - accuracy: 0.7830 - val_loss: 0.5765 - val_accuracy: 0.8253
Epoch 2/2
10875/10875 [==============================] - 6236s 573ms/step - loss: 0.4508 - accuracy: 0.8644 - val_loss: 0.4860 - val_accuracy: 0.8548


In [22]:
#Vérifier les 5 classes dans lequelles ont se trompent le plus
learner.view_top_losses(n=5, preproc=trans)

----------
id:41563 | loss:11.82 | true:7 | pred:3)

----------
id:20103 | loss:11.17 | true:16 | pred:19)

----------
id:34485 | loss:10.52 | true:4 | pred:6)

----------
id:2979 | loss:10.51 | true:12 | pred:22)

----------
id:25126 | loss:10.26 | true:7 | pred:19)



 <h5 style="color:blue">III.4 Evaluation des métrics</h2> 



In [21]:
#Fonction permettant d'affficher les différentes valeurs du classiffieur pour des metrics et affihage de la matrice de confusion
learner.validate(class_names=categories)

              precision    recall  f1-score   support

          19       0.90      0.93      0.92     13947
           9       0.83      0.79      0.81       624
          24       0.78      0.69      0.73      1175
          22       0.82      0.76      0.79      2120
           6       0.78      0.82      0.80      2429
           3       0.69      0.66      0.67      1799
          14       0.90      0.85      0.87      2509
          26       0.90      0.91      0.91      3812
          13       0.74      0.77      0.75       850
           5       0.84      0.86      0.85       917
          11       0.78      0.75      0.77      2329
          17       0.81      0.67      0.73       307
           4       0.67      0.71      0.69       144
          20       0.89      0.91      0.90      2953
           8       0.79      0.80      0.80      1302
          18       0.88      0.85      0.86       790
          27       0.83      0.85      0.84       434
          10       0.79    

array([[13030,    14,    49,   124,    98,   104,    53,    86,    31,
           23,    93,     6,     3,    23,    96,    27,    13,     0,
            0,    14,     7,    35,     1,     1,     1,    15,     0,
            0],
       [   22,   491,    10,     3,    24,    15,     4,    29,     2,
            2,     3,     0,     1,     6,     1,     0,     0,     0,
            4,     0,     0,     1,     1,     0,     0,     5,     0,
            0],
       [   73,    10,   806,     4,    11,    17,     0,    22,   154,
           10,     1,     0,     0,    21,     4,     0,     1,     2,
            5,     4,     1,     5,     0,     0,    21,     2,     1,
            0],
       [  310,     2,     4,  1608,    27,    48,    32,    18,     2,
            4,    16,     2,     4,     7,     6,     1,     6,     1,
            1,     1,     2,     6,     1,     5,     1,     5,     0,
            0],
       [  122,    11,    10,    17,  1989,    44,     9,    43,    14,
            7

**Score Macro : 81%**

In [21]:
pred=pd.read_csv("C:/Users/konate/Desktop/IA_kaagle/seventh_k_submission.csv")

In [ ]:
def macro_disparate_impact(people):
    counts = people.groupby(['Category', 'gender']).size().unstack('gender')
    counts['disparate_impact'] = counts[['M', 'F']].max(axis='columns') / counts[['M', 'F']].min(axis='columns')
    return counts['disparate_impact'].mean()



In [32]:
y_pred =pred.join(categories.set_index("Category"), on = "Category")
genders_test = pd.read_json(path+'/test.json').set_index('Id')['gender']
test_people = pd.concat((y_pred, genders_test), axis='columns')


In [33]:
print('macro disparate impact :',macro_disparate_impact(test_people))

macro disparate impact : 4.493004982100444
